In [2]:
import pandas as pd
import glob
import numpy as np
import os, sys
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

In [8]:
def append_cols(df1,df2,task,cols):
    for col in cols:
        df1['{}_{}'.format(task,col)] = df2.loc[df2['subject'] == df1['subject'],col]
    
    return df1

In [18]:
#create all data frames
#root dir for mac
#root_dir = "/Users/govild2/National Institutes of Health/NIMH CDN lab - Documents/Experiments/1-Data/IDM/IDM-CloudResearch/"
#root dir for windows
root_dir = "C:/Users/govild2/National Institutes of Health/NIMH CDN lab - Documents/Datasets/IDM/IDM-CloudResearch"
conf_dir = os.path.join(root_dir,"Confidence_analysis/CASENDRE_output")
value_dir = os.path.join(root_dir,"IDM_Model_Output")
save_dir = os.path.join(root_dir,"Confidence_analysis")

tasks = ['crdm','cdd','cpdm_block_1','cpdm_block_2','cpdm_block_3','cpdm_block_4']
batches = ["Batch_1_2","Batch_3","Batch_4"]

df_batches = []

#get meta-uncertainty values estimated by the CASANDRE model
for task in tasks:
    frames = []
    for batch in batches:
        idf = pd.read_csv(os.path.join(conf_dir,batch,"{}_metauncertainty_by_subject.csv".format(task)))
        frames = frames + [idf]
    df_batch = pd.concat(frames,ignore_index=True)
    df_batches = df_batches + [df_batch]

[df_crdm_batches,df_cdd_batches,df_cpdm_b1_batches,df_cpdm_b2_batches,df_cpdm_b3_batches,df_cpdm_b4_batches] = df_batches

#get parameters from value-based model
tasks = ['CRDM_analysis','CDD_analysis','CDD_analysis_alpha']
df_batches = []
for task in tasks:
    frames = []
    for batch in batches:
        idf = pd.read_csv(os.path.join(value_dir,batch,"{}_{}.csv".format(batch,task)))
        frames = frames + [idf]
    df_batch = pd.concat(frames,ignore_index=True)
    df_batches = df_batches + [df_batch]

[df_total_CRDM_analysis,df_total_CDD_analysis,df_total_CDD_analysis_alpha] = df_batches
print(list(df_total_CDD_analysis))

['Unnamed: 0', 'subject', 'task', 'response_rate', 'percent_impulse', 'negLL', 'gamma', 'kappa', 'alpha', 'at_bound', 'LL', 'LL0', 'AIC', 'BIC', 'R2', 'correct', 'prob_span', 'fig_fn']


In [21]:
#combine dfs
#cdd
df_aggregate = df_cdd_batches
df_aggregate = df_aggregate.rename(columns={'meta_uncertainty':'cdd_meta'})

#crdm

df_aggregate['crdm_meta'] = df_crdm_batches.loc[df_crdm_batches['subject']==df_aggregate['subject'],'meta_uncertainty']

#cpdm
#cpdm block 1
df_aggregate['cpdm_b1_meta'] = df_cpdm_b1_batches.loc[df_cpdm_b1_batches['subject']==df_aggregate['subject'],'meta_uncertainty']

#cpdm block 2
df_aggregate['cpdm_b2_meta'] = df_cpdm_b2_batches.loc[df_cpdm_b2_batches['subject']==df_aggregate['subject'],'meta_uncertainty']

#cpdm block 3
df_aggregate['cpdm_b3_meta'] = df_cpdm_b3_batches.loc[df_cpdm_b3_batches['subject']==df_aggregate['subject'],'meta_uncertainty']

#cpdm block 4
df_aggregate['cpdm_b4_meta'] = df_cpdm_b4_batches.loc[df_cpdm_b4_batches['subject']==df_aggregate['subject'],'meta_uncertainty']

#computing average across 4 cpdm blocks
cols = ['cpdm_b1_meta','cpdm_b2_meta','cpdm_b3_meta','cpdm_b4_meta']
df_aggregate['cpdm_avg_meta'] = np.mean(df_aggregate[cols],axis=1)
#sorting by subject
df_aggregate = df_aggregate.sort_values(by=['subject']).reset_index(drop=True)

#CRDM analyses (beta parameter)
#print(df_total_CRDM_analysis)
cols = ['negLL', 'gamma', 'beta', 'alpha', 'R2']
task = 'crdm'
df_total_CRDM_analysis = df_total_CRDM_analysis.sort_values(by=['subject']).reset_index(drop=True)
df_aggregate = append_cols(df_aggregate,df_total_CRDM_analysis,task,cols)

cols = ['negLL', 'gamma', 'kappa', 'alpha', 'R2']
task = 'cdd'
df_total_CDD_analysis = df_total_CDD_analysis.sort_values(by=['subject']).reset_index(drop=True)
df_aggregate = append_cols(df_aggregate,df_total_CDD_analysis,task,cols)

task = 'cdd_alpha'
df_total_CDD_analysis_alpha = df_total_CDD_analysis_alpha.sort_values(by=['subject']).reset_index(drop=True)
df_aggregate = append_cols(df_aggregate,df_total_CDD_analysis_alpha,task,cols)

fn = os.path.join(save_dir,"Model_Parameters_Aggregate.csv")
print("Saving Aggregate Data to: {}".format(fn))
df_aggregate.to_csv(fn,index=False)

Saving Aggregate Data to: C:/Users/govild2/National Institutes of Health/NIMH CDN lab - Documents/Datasets/IDM/IDM-CloudResearch\Confidence_analysis\Model_Parameters_Aggregate.csv
